### Setup

In [108]:
import pandas as pd
import altair as alt
from functools import partial

import importlib
import utils
# Cache-Probleme umgehen
importlib.reload(utils)
from utils import create_highlight_func
from utils import highlight_rows

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [142]:
# Farbgebung in Diagrammen
color1 = '#06507F'  # Dunkelblau
color2 = '#330241'  # Dunkellila

### Data Load

In [3]:
path_data = 'https://raw.githubusercontent.com/mm391-030401/project/refs/heads/main/data/processed/'
file_data= 'data_final.csv'

data = pd.read_csv(path_data + file_data, sep=',', encoding='utf-8')

# Obwohl diese Spalten bereits im Notebook "02_MM_convert_columns" zu Strings konvertiert wurden, wurden sie beim erneuten Einlesen wieder als Integer erkannt
# Allerdings ist es sinnvoll, diese Spalten nicht als Integer (diskret) zu betrachten 
# sondern als ordinale Variablen (Rechenoperationen sind nicht sinnig für die Zeitangaben)
to_str_cols = ['monate_seit_einfuehrung_programm_kohorte', 'monat', 'monat_jahr',
       'monat_monat', 'kohorte', 'kohorte_jahr', 'kohorte_monat', 'monate_seit_existenz_kohorte']

# Konvertierung der Spalten in string 
for col in to_str_cols: 
    data[col] = data[col].astype(str)

### Variable List

In [5]:
y_label = 'retentionrate'
list_numeric = data.select_dtypes(include = 'number').columns.to_list()
list_category = data.select_dtypes(include = ['object', 'bool']).columns.to_list()

### Data Structure

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 703 entries, 0 to 702
Data columns (total 15 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   monate_seit_einfuehrung_programm_kohorte  703 non-null    object 
 1   monat                                     703 non-null    object 
 2   monat_jahr                                703 non-null    object 
 3   monat_monat                               703 non-null    object 
 4   monat_jahreszeit                          703 non-null    object 
 5   kohorte                                   703 non-null    object 
 6   kohorte_jahr                              703 non-null    object 
 7   kohorte_monat                             703 non-null    object 
 8   kohorte_jahreszeit                        703 non-null    object 
 9   erster_monat_kohorte_fg                   703 non-null    bool   
 10  monate_seit_existenz_kohorte          

In [10]:
data.shape

(703, 15)

In [11]:
data.head()

,monate_seit_einfuehrung_programm_kohorte,monat,monat_jahr,monat_monat,monat_jahreszeit,kohorte,kohorte_jahr,kohorte_monat,kohorte_jahreszeit,erster_monat_kohorte_fg,monate_seit_existenz_kohorte,kohortengroesse_indexiert,identifizierte_kunden_indexiert,rabatt_indexiert,retentionrate
0,-2,201408,2014,8,Sommer,201408,2014,8,Sommer,True,0,0.41,0.41,2.54,100.000000
1,-2,201409,2014,9,Herbst,201408,2014,8,Sommer,False,1,0.41,0.40,7.14,97.560976
2,-2,201410,2014,10,Herbst,201408,2014,8,Sommer,False,2,0.41,0.39,9.28,95.121951
3,-2,201411,2014,11,Herbst,201408,2014,8,Sommer,False,3,0.41,0.38,3.22,92.682927
4,-2,201412,2014,12,Winter,201408,2014,8,Sommer,False,4,0.41,0.38,7.15,92.682927


In [12]:
data.tail()

,monate_seit_einfuehrung_programm_kohorte,monat,monat_jahr,monat_monat,monat_jahreszeit,kohorte,kohorte_jahr,kohorte_monat,kohorte_jahreszeit,erster_monat_kohorte_fg,monate_seit_existenz_kohorte,kohortengroesse_indexiert,identifizierte_kunden_indexiert,rabatt_indexiert,retentionrate
698,32,201707,2017,7,Sommer,201706,2017,6,Sommer,False,1,18.36,9.45,17.88,51.470588
699,32,201708,2017,8,Sommer,201706,2017,6,Sommer,False,2,18.36,8.40,10.59,45.751634
700,33,201707,2017,7,Sommer,201707,2017,7,Sommer,True,0,17.49,17.49,23.76,100.000000
701,33,201708,2017,8,Sommer,201707,2017,7,Sommer,False,1,17.49,8.54,11.85,48.827902
702,34,201708,2017,8,Sommer,201708,2017,8,Sommer,True,0,15.13,15.13,14.28,100.000000


### Analysis

#### Descriptive Statistics

In [6]:
data[list_numeric].describe().T

,count,mean,std,min,25%,50%,75%,max
kohortengroesse_indexiert,703.0,47.926743,77.462511,0.410000,23.110000,24.850000,33.930000,375.13
identifizierte_kunden_indexiert,703.0,28.301565,53.452328,0.310000,10.890000,12.320000,21.215000,375.13
rabatt_indexiert,703.0,99.719090,257.969362,2.260000,21.500000,33.020000,61.045000,2570.59
retentionrate,703.0,55.442768,15.491696,35.752785,46.394922,49.102845,57.336771,100.00


In [7]:
data[list_category].describe().T

,count,unique,top,freq
monate_seit_einfuehrung_programm_kohorte,703,37,-2,37
monat,703,37,201708,37
monat_jahr,703,4,2016,282
monat_monat,703,12,8,76
monat_jahreszeit,703,4,Sommer,217
kohorte,703,37,201408,37
kohorte_jahr,703,4,2015,318
kohorte_monat,703,12,8,76
kohorte_jahreszeit,703,4,Herbst,207
erster_monat_kohorte_fg,703,2,False,666


- `kohortengroesse_indexiert` sowie `identifizierte_kunden_indexiert` zeigen eine hohe Diskrepanz zwischen dem 3. Quantil und dem Maximalwert. Es gibt also wenige Kohorten, die eine sehr hohe Anzahl an Kunden ausweisen. Je höher die anfängliche Kundenanzahl desto höher dann auch die Anzahl der möglichen Kunden, die sich jeden Monat identifizieren können.
- Ebenfalls gibt es bestimmte Kohorten-Monats-Kombinationen, die einen hohen Rabatt erhalten haben. 
- Die maximale Retentionrate liegt bei 100. Im ersten Monat der Kohorte (dann, wenn die Kohorte entsteht) ist der Wert von Spalte `kohortengroesse_indexiert` und `identifizierte_kunden_indexiert` immer gleich. Hier liegt also die Retentionrate bei 100. Der starke Kundenrückgang nach dem ersten Monat einer Kohorte zeigt sich in den Werten von den Quantilen. Die Differenz zwischen 3. Quantil und Maximalwert ist ebenfalls hoch.
- Keine der Spalten enthält missing values.

#### Numeric Data

In [143]:
alt.Chart(data).mark_circle(color = color1).encode(
    x=alt.X(alt.repeat("column"), 
            type='quantitative',
            scale=alt.Scale(zero=False)
            ),
    y=alt.Y(alt.repeat("row"), 
            type='quantitative',
            scale=alt.Scale(zero=False)
             )
).properties(
    width=150,
    height=150
).repeat(
    row=list_numeric,
    column=list_numeric
)

alt.RepeatChart(...)

Da die identifizierten Kunden und die Kohortengröße zur Berechnung der Response Variable benötigt werden, eignen sie sich nicht als Predictor. Der Rabatt könnte einen Eunfluss haben. Das Muster wird nachfolgend genauer untersucht.

In [150]:
# Variable anlegen für den Rabatt
x_label = 'rabatt_indexiert'

In [151]:
alt.Chart(data).mark_circle(color=color1).encode(
    x = alt.X(x_label),
    y = alt.Y(y_label),
    tooltip = ['kohorte', 'monat']
).interactive()

alt.Chart(...)

Die rechten Ausreißer stammen ausschließlich aus der Kohorte 202410. Die Kohorte beansprucht den meisten Rabatt. Damit ein möglicher Zusammenhang nicht nur durch die oberen Ausreißer erscheint, wird sich der Scatterplot nochmal nur für diese Kohorte und einmal für alle ohne diese Kohorte angeschaut.

In [152]:
data_201410 = data[data['kohorte'] == '201410']

alt.Chart(data_201410).mark_circle(color=color1).encode(
    x = alt.X(x_label),
    y = alt.Y(y_label),
    tooltip = ['kohorte', 'monat']
).interactive()

alt.Chart(...)

In [153]:
data_wo_201410 = data[data['kohorte'] != '201410']

alt.Chart(data_wo_201410).mark_circle(color=color1).encode(
    x = alt.X(x_label),
    y = alt.Y(y_label),
    tooltip = ['kohorte', 'monat']
).interactive()

alt.Chart(...)

Nach Betrachtung der Daten wirkt es fast so, als ob jede Kohorte seinem eigenem Muster folgt. Daher wird einmal für jede Kohorte ein Scatterplot erstellt.

In [17]:
# Liste der einzigartigen Werte in der Spalte 'kohorte'
unique_kohorten = data['kohorte'].unique()

In [154]:
# Leere Liste, um die Charts zu speichern
charts = []

# Schleife über alle einzigartigen Werte in 'kohorte'
for kohorte in unique_kohorten:
    # Filtert den DataFrame nach der aktuellen Kohorte
    df_filtered = data[data['kohorte'] == kohorte]
    
    # Erstellt das Diagramm
    chart = alt.Chart(df_filtered).mark_circle(color=color1).encode(
        x=alt.X(x_label),
        y=alt.Y(y_label),
        tooltip=['kohorte', 'monat', y_label, x_label]
    ).interactive()
    
    # Fügt eine Überschrift hinzu
    chart = chart.properties(
        title=f'Kohorte {kohorte}',
        width = 150,
        height = 150
    )
    
    # Fügt das Diagramm der Liste hinzu
    charts.append(chart)

# Erstellt ein Rasterlayout aus den Diagrammen
grid_chart = alt.concat(*charts, columns=6)

grid_chart.display()

alt.ConcatChart(...)

Bei den meisten Kohorten scheint eine positive Korrelation zwischen dem Rabatt und der Retentionrate zu exisitieren. Dies wird nun mit einer Berechnung der Korrelation je Kohorte überprüft.

In [155]:
# Leere Liste, um die Ergebnisse zu speichern
korrelations_liste = []

# Schleife über alle einzigartigen Werte in 'kohorte'
for kohorte in unique_kohorten:
    # Filtert den DataFrame nach der aktuellen Kohorte
    df_filtered = data[data['kohorte'] == kohorte]

    # Berechnet die Korrelation (Es wird Spearman verwendet, da bereits in der deskriptven Statistik sowie in den Scatterplots deutlich wird, dass es Ausreißer gibt.)
    # Spearman ist besser im Umgang mit Ausreißern im Vergleich zu Pearson
    korrelation = df_filtered[[y_label, x_label]].corr(method='spearman').iloc[0, 1]
    
    # Fügt die Ergebnisse zur Liste hinzu
    korrelations_liste.append({
        'Kohorte': kohorte,
        'Korrelation': korrelation
    })

# Erstellt einen DataFrame aus der Liste
korrelations_df = pd.DataFrame(korrelations_liste)

# Anzahl der Zeilen
n = len(korrelations_df)

# Aufteilen des DataFrames in zwei Hälften
first_half = korrelations_df.iloc[:n//2]
second_half = korrelations_df.iloc[n//2:]

# Kombiniert die beiden Hälften in einem DataFrame mit zwei Spalten
# So soll das Df dann später übersichtlicher und kompakter dargestellt werden
combined_df = pd.concat([first_half.reset_index(drop=True), second_half.reset_index(drop=True)], axis=1)
combined_df.columns = ['Kohorte_Zeile1', 'Korrelation_Zeile1', 'Kohorte_Zeile2', 'Korrelation_Zeile2']

# Transponiert das df
combined_df = combined_df.T

# Anwenden der Farbformatierung und Runden der Werte auf zwei Nachkommastellen
highlight_func = create_highlight_func(combined_df, color1, color2)
styled_df = combined_df.style.apply(highlight_func, axis=1).format(precision=2)
styled_df


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Kohorte_Zeile1,201408,201409,201410,201411,201412,201501,201502,201503,201504,201505,201506,201507,201508,201509,201510,201511,201512,201601,nan
Korrelation_Zeile1,0.08,0.19,0.65,0.64,0.69,0.42,0.30,0.22,0.12,0.06,0.16,0.02,-0.16,-0.29,-0.36,-0.32,-0.06,-0.14,nan
Kohorte_Zeile2,201602,201603,201604,201605,201606,201607,201608,201609,201610,201611,201612,201701,201702,201703,201704,201705,201706,201707,201708
Korrelation_Zeile2,0.04,-0.05,0.21,0.24,0.46,0.43,0.69,0.80,0.91,0.94,0.83,0.93,0.89,0.94,0.90,1.00,1.00,1.00,nan


Die Korrelation zwischen dem Rabatt und der Retentionrate schwankt je nach Kohorte. Es ist fraglich, ob der Rabatt tatsächlich ein geeigneter Predictor ist.  
Daher wird vorgeschlagen erst die Auswahl der Zeitvariablen zu optimieren und dann am Schluss den Rabatt als Feature mitzugeben und zu schauen, ob es das Modell positiv beeinflusst.

#### Categoric Data

In [157]:
charts_cat = []

for cat in list_category:
    # Erstellen einer Kopie des df für die Verarbeitung
    temp_data = data.copy()
    if temp_data[cat].dtype == bool:
        temp_data[cat] = temp_data[cat].astype(str)  # Wandelt boolesche Werte in Strings um
    
    # Prüfen, ob die Kategorie numerisch sortiert werden soll
    try:
        # Versuche, die Kategorie-Werte in numerische Form zu bringen
        sort_order = sorted(temp_data[cat].unique(), key=lambda x: float(x))
    except ValueError:
        # Wenn nicht numerisch, lasse die Sortierung standardmäßig
        sort_order = None
    
    # Erstellen des Diagramms
    chart = alt.Chart(temp_data).mark_bar(color=color1).encode(
        x=alt.X(f'mean({y_label}):Q', title=y_label),
        y=alt.Y(f'{cat}', title=cat, sort=sort_order),
        tooltip=[
        alt.Tooltip(f'{cat}:N', title='Kategorie'),  # Kategorie anzeigen
        alt.Tooltip(f'mean({y_label}):Q', title='Mittelwert', format=".2f"),  # Mittelwert mit 2 Dezimalstellen
        alt.Tooltip(f'count():Q', title='Anzahl Zeilen für Mittelwert') # Wie viele Werte sind in den Mittelwert eingeflossen?
    ]
    ).properties(
        title=f'Mean {y_label} je {cat}',
        width=300,
        height=300
    ).interactive()
    
    charts_cat.append(chart)

grid_chart_cat = alt.concat(*charts_cat, columns=3)
grid_chart_cat.display()


alt.ConcatChart(...)

- Die Variable `erster_monat_kohorte_fg` scheint relevant für das Modell zu sein. So wird deutlich, dass der erste Monat einer Kohorte, wo die Retention immer bei 100 liegt, nicht die Standardretention ist.
- Ebenfalls wird deutlich, dass beiden zeitliche Faktoren (Entstehung der Kohorte und Zeitverlauf des Einkaufsverhaltens) in das Modell inkludiert werden sollten. Im Zeitverlauf (`monat`) wird die Retention eher schlechter, während frühe als auch spätere Kohorten (`kohorte`) eher eine bessere Retention zeigen. Es besteht ein Unterschied in der Reaktion der Retention zwischen dem normalen Zeitverlauf in Monaten vs. der Monate der Entstehung der Kohorten. Das "Level" der Zeit (also braucht man die Granularität von `monat` und `kohorte` oder reicht z.B. aich `kohorte_monat` und `monat_monat`) sollte in mehreren Testläufen optimiert werden.
- Die beiden Spalten `monate_seit_einfuehrung_programm_kohorte` und `monate_seit_existenz_kohorte` sind davon seperat zu betrachten. `monate_seit_einfuehrung_programm_kohorte` gibt an, an welchem Zeitpunkt des Lebenszyklus des Programms die Kohorte dazugekommen ist. Die Kohorten, die schon relativ früh ins Programm eingetreten sind, zeigen eine erhöhte Retention Rate. Auch die sehr späten Kohorten zeigen eine hohe Retention Rate. Dies liegt jedoch daran, dass nur wenige Werte in die Mittelwertberechnung eingeflossen sind (siehe Tooltip). Da der erste Monat immer eine Retentionrate von 100 aufweist, wirkt es so, als ob die späteren Kohorten eine hohe Retention haben. Würde man diese Kohorten im weiteren Zeitverlauf beobachten, so würde die Retentionrate analog der anderen Kohorten ebenfalls sinken. Die Spalte `monate_seit_einfuehrung_programm_kohorte` ist analog zur Spalte `kohorte`.  
`monate_seit_existenz_kohorte` gibt an, wie lang die aktuelle Kohorte schon teil des Programms ist. Daher ist bei Wert 0 die Retentionrate auch immer 100, da sie den ersten Monat der Kohorte angibt und auch alle 37 Kohorten beinhaltet. Der rapide Abstieg von 0 auf 1 zeigt das normale Verhalten der Kunden, dass die, die im ersten Monat da waren nicht alle im Folgemonat zurückkehren. Bis Monat 6 ist ein Abstieg der Retentionrate zu beobachten. Danach steigt die Retention tendenziell wieder. Dies zeigt zwei Effekte: Es findet keine Kundenabwanderung im großen Stil mehr statt. Gleichzeitig kommen eher die zurück, die im Monat 0 sich das erste mal identifiziert haben (Schwankendes Einkaufsverhalten, welches der Kunde über die App identifiziert). Zusätzlich sind immer weniger Kohorten vorhanden, die den Mittelwert beeinflussen, je stärker diese Variable ansteigt (Wert 36 zeigt ausschließlich die älteste Kohorte). Aus der Betrachtung der Variable `kohorte` geht bereits hervor, dass frühere Kohorten eine tendenziell höhere Retention zeigen. Der Doppeleffekt führt zu einer zunehmenden Steigerung der Retentiorate. Diese Variable spiegelt sich nicht in einer anderen wieder. Sie scheint relevant zu sein. Allerdings ist zu prüfen, ob die Granualirität notwendig ist oder es lieber in 5er Schritten zusammengefasst werden sollte.